# Testing the probability that Kurtz' theorem is satisfied

I.e., testing the conditions of the theorem, when given a real rooted polynomial.

Tested using Julia 1.9.4 with the packages Oscar 1.0.5, Distributions 0.25.

Loading packages:

In [1]:
using Oscar, Random, Distributions

Defining $\mathbb{R}[x]$:

In [2]:
RR = RealField()
R, x = polynomial_ring(RR, :x)

(Univariate polynomial ring in x over RR, x)

## Functions

### probability_satisfied
Function to sample real rooted polynomials and check how many of them satisfy Kurtz' theorem (note that this function outputs a tuple (number satisfied, sample size)):

In [3]:
function probability_satisfied(degree::Int, sample_interval::Tuple{BigFloat, BigFloat}, sample_size::Int=100, rng_seed=123)
    """
    Returns ratio of polynomials satisfying the conditions of [Kurtz1992].
    To do this, the function generates polynomials with uniformly distributed real roots and checks if they satisfy the conditions of [Kurtz1992, Thm 1].
    Input:
        - degree: The degree of the polynomials, which we wish to test.
        - sample_interval: Interval the roots should be sampled from.
        - sample_size=100: How many polynomials to sample. Default is 100.
        - rng_seed=123: Seed for our random device.
    Output: (satisfied, sample_size)
    """
    # Setup random generator:
    rng = Xoshiro(rng_seed)

    # Uniform distribution to sample from:
    sample_space = Distributions.Uniform(sample_interval[1], sample_interval[2])

    # Initializing variable to count number of polynomials, which satisfy the conditions in the theorem:
    number_satisfying_conditions = 0

    # Perform sampling:
    for i in 1:sample_size
        # Generate polynomial:
        random_polynomial = generate_random_polynomial(degree, sample_space)
        # Check if it satisfies conditions, and if so, increment counter by 1:
        satisfies = check_conditions(random_polynomial)[1]
        if satisfies
            number_satisfying_conditions += 1
        end
    end

    return number_satisfying_conditions/sample_size
end

probability_satisfied (generic function with 3 methods)

### check_conditions
Function to check that conditions of Kurtz' theorem are satisfied. It does not check for real roots by default, as it assumes the polynomial is constructed, such that the roots are real.

In [4]:
function check_conditions(pol::RealPoly, check_roots_real::Bool = false)
    """
    Determines if all conditions are satisfied:
        - All coefficients are positive.
        - S(P,i)>4, for all i=1,...,n-1
        - The polynomial has some non-real roots. (This last check only happens if check_roots_real is true.)
    Input:
        - RealPoly pol
        - Bool check_roots_real
    """
    coeffs = collect(coefficients(pol)) # note: index 1 is constant term, index n+1 is n'th term
    pol_degree = Oscar.degree(pol)
    
    # Check that all coefficients are positive:
    for a in coeffs
        if a <= 0
            return (false, "Not all coefficients are positive.")
        end
    end
    
    # Check that S(P,i)>4 for all i:
    for i in 1:(pol_degree - 1)
        S = (coeffs[i+1]^2)/(coeffs[(i-1)+1]*coeffs[(i+1)+1])
        if S <= 4
            return (false, "S(pol, " * string(i) * ")<=4")
        end
    end

    # If check_roots_real==true: Check that the polynomial has only real roots:
    if check_roots_real==true
        coeffs = collect(coefficients(pol))
        CC = ComplexField()
        C, z = CC[:z]
        fC = C(coeffs)
        complex_roots = roots(fC)
        for root in complex_roots
            if !isreal(root)
                return (false, "Some roots are non-real.")
            end
        end
    end
    
    return (true, "All conditions are satisfied.")
end

check_conditions (generic function with 2 methods)

### generate_random_polynomial
Function to generate the polynomials:

In [5]:
function generate_random_polynomial(degree::Int, sample_space::Uniform{BigFloat})
    """
    Returns polynomial generated from uniformly random roots.
    Input:
        - degree: degree of the desired polynomial.
        - sample_space: uniform distribution to sample from.
    Output: polynomial
    """
    pol = 1
    for i in 1:degree
        root = rand(sample_space)
        pol *= (x-root)
    end

    return pol
end

generate_random_polynomial (generic function with 1 method)

## Tests

### Degree 3:

In [6]:
print("Interval (-100,0): ")
@time println(probability_satisfied(3, (BigFloat(-100.0), BigFloat(0.0)), 100000000))
print("Interval (-1000,0): ")
@time println(probability_satisfied(3, (BigFloat(-1000.0), BigFloat(0.0)), 100000000))
print("Interval (-10000,0): ")
@time println(probability_satisfied(3, (BigFloat(-10000.0), BigFloat(0.0)), 100000000))
print("Interval (-100000,0): ")
@time println(probability_satisfied(3, (BigFloat(-100000.0), BigFloat(0.0)), 100000000))
print("Interval (-1000000,0): ")
@time println(probability_satisfied(3, (BigFloat(-1000000.0), BigFloat(0.0)), 100000000))

Interval (-100,0): 0.11938581
673.590053 seconds (6.35 G allocations: 398.412 GiB, 23.96% gc time, 0.02% compilation time)
Interval (-1000,0): 0.11937622
667.734507 seconds (6.35 G allocations: 398.408 GiB, 24.07% gc time)
Interval (-10000,0): 0.11934174
672.859719 seconds (6.35 G allocations: 398.409 GiB, 23.74% gc time)
Interval (-100000,0): 0.11938009
691.138539 seconds (6.44 G allocations: 409.401 GiB, 23.33% gc time)
Interval (-1000000,0): 0.11938332
691.342360 seconds (6.45 G allocations: 411.767 GiB, 23.29% gc time)


In [7]:
print("Interval (-100,-1): ")
@time println(probability_satisfied(3, (BigFloat(-100.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000,-1): ")
@time println(probability_satisfied(3, (BigFloat(-1000.0), BigFloat(-1.0)), 100000000))
print("Interval (-10000,-1): ")
@time println(probability_satisfied(3, (BigFloat(-10000.0), BigFloat(-1.0)), 100000000))
print("Interval (-100000,-1): ")
@time println(probability_satisfied(3, (BigFloat(-100000.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000000,-1): ")
@time println(probability_satisfied(3, (BigFloat(-1000000.0), BigFloat(-1.0)), 100000000))

Interval (-100,-1): 0.09765286
670.851662 seconds (6.35 G allocations: 397.847 GiB, 23.69% gc time)
Interval (-1000,-1): 0.11685948
666.905854 seconds (6.36 G allocations: 399.320 GiB, 23.77% gc time)
Interval (-10000,-1): 0.11908152
694.095533 seconds (6.45 G allocations: 411.653 GiB, 22.85% gc time)
Interval (-100000,-1): 0.1193003
689.169412 seconds (6.45 G allocations: 411.815 GiB, 23.00% gc time)
Interval (-1000000,-1): 0.11941458
687.005810 seconds (6.45 G allocations: 411.822 GiB, 23.09% gc time)


### Degree 4:

In [8]:
print("Interval (-100,0): ")
@time println(probability_satisfied(4, (BigFloat(-100.0), BigFloat(0.0)), 100000000))
print("Interval (-1000,0): ")
@time println(probability_satisfied(4, (BigFloat(-1000.0), BigFloat(0.0)), 100000000))
print("Interval (-10000,0): ")
@time println(probability_satisfied(4, (BigFloat(-10000.0), BigFloat(0.0)), 100000000))
print("Interval (-100000,0): ")
@time println(probability_satisfied(4, (BigFloat(-100000.0), BigFloat(0.0)), 100000000))
print("Interval (-1000000,0): ")
@time println(probability_satisfied(4, (BigFloat(-1000000.0), BigFloat(0.0)), 100000000))

Interval (-100,0): 0.00486293
841.055325 seconds (7.95 G allocations: 512.813 GiB, 22.59% gc time)
Interval (-1000,0): 0.00485887
835.986263 seconds (7.95 G allocations: 512.814 GiB, 22.63% gc time)
Interval (-10000,0): 0.00487097
854.815077 seconds (7.95 G allocations: 512.891 GiB, 22.38% gc time)
Interval (-100000,0): 0.00485577
867.007454 seconds (8.05 G allocations: 525.395 GiB, 21.97% gc time)
Interval (-1000000,0): 0.00487046
869.574618 seconds (8.05 G allocations: 526.219 GiB, 21.92% gc time)


In [9]:
print("Interval (-100,-1): ")
@time println(probability_satisfied(4, (BigFloat(-100.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000,-1): ")
@time println(probability_satisfied(4, (BigFloat(-1000.0), BigFloat(-1.0)), 100000000))
print("Interval (-10000,-1): ")
@time println(probability_satisfied(4, (BigFloat(-10000.0), BigFloat(-1.0)), 100000000))
print("Interval (-100000,-1): ")
@time println(probability_satisfied(4, (BigFloat(-100000.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000000,-1): ")
@time println(probability_satisfied(4, (BigFloat(-1000000.0), BigFloat(-1.0)), 100000000))

Interval (-100,-1): 0.00157997
841.331661 seconds (7.93 G allocations: 511.256 GiB, 22.48% gc time)
Interval (-1000,-1): 0.00442414
852.738406 seconds (7.98 G allocations: 516.284 GiB, 22.48% gc time)
Interval (-10000,-1): 0.00481732
875.632216 seconds (8.05 G allocations: 526.190 GiB, 21.66% gc time)
Interval (-100000,-1): 0.00485834
867.703395 seconds (8.05 G allocations: 526.225 GiB, 21.74% gc time)
Interval (-1000000,-1): 0.00486441
864.545923 seconds (8.05 G allocations: 526.225 GiB, 21.83% gc time)


### Degree 5:

In [10]:
print("Interval (-100,0): ")
@time println(probability_satisfied(5, (BigFloat(-100.0), BigFloat(0.0)), 100000000))
print("Interval (-1000,0): ")
@time println(probability_satisfied(5, (BigFloat(-1000.0), BigFloat(0.0)), 100000000))
print("Interval (-10000,0): ")
@time println(probability_satisfied(5, (BigFloat(-10000.0), BigFloat(0.0)), 100000000))
print("Interval (-100000,0): ")
@time println(probability_satisfied(5, (BigFloat(-100000.0), BigFloat(0.0)), 100000000))
print("Interval (-1000000,0): ")
@time println(probability_satisfied(5, (BigFloat(-1000000.0), BigFloat(0.0)), 100000000))

Interval (-100,0): 5.144e-5
1020.952187 seconds (9.52 G allocations: 629.432 GiB, 21.74% gc time)
Interval (-1000,0): 5.231e-5
1020.426620 seconds (9.52 G allocations: 629.429 GiB, 21.88% gc time)
Interval (-10000,0): 5.08e-5
1033.431820 seconds (9.53 G allocations: 630.261 GiB, 21.59% gc time)
Interval (-100000,0): 5.278e-5
1039.337069 seconds (9.62 G allocations: 642.595 GiB, 21.16% gc time)
Interval (-1000000,0): 5.13e-5
1042.313971 seconds (9.62 G allocations: 642.839 GiB, 21.06% gc time)


In [11]:
print("Interval (-100,-1): ")
@time println(probability_satisfied(5, (BigFloat(-100.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000,-1): ")
@time println(probability_satisfied(5, (BigFloat(-1000.0), BigFloat(-1.0)), 100000000))
print("Interval (-10000,-1): ")
@time println(probability_satisfied(5, (BigFloat(-10000.0), BigFloat(-1.0)), 100000000))
print("Interval (-100000,-1): ")
@time println(probability_satisfied(5, (BigFloat(-100000.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000000,-1): ")
@time println(probability_satisfied(5, (BigFloat(-1000000.0), BigFloat(-1.0)), 100000000))

Interval (-100,-1): 0.0
1028.367650 seconds (9.50 G allocations: 627.521 GiB, 21.60% gc time)
Interval (-1000,-1): 3.061e-5
1039.765472 seconds (9.57 G allocations: 636.104 GiB, 21.49% gc time)
Interval (-10000,-1): 4.943e-5
1063.133125 seconds (9.62 G allocations: 642.816 GiB, 20.82% gc time)
Interval (-100000,-1): 5.111e-5
1052.997061 seconds (9.62 G allocations: 642.839 GiB, 20.94% gc time)
Interval (-1000000,-1): 5.016e-5
1050.952301 seconds (9.62 G allocations: 642.839 GiB, 21.04% gc time)


### Degree 6:

In [12]:
print("Interval (-100,0): ")
println(probability_satisfied(6, (BigFloat(-100.0), BigFloat(0.0)), 100000000))
print("Interval (-1000,0): ")
println(probability_satisfied(6, (BigFloat(-1000.0), BigFloat(0.0)), 100000000))
print("Interval (-10000,0): ")
println(probability_satisfied(6, (BigFloat(-10000.0), BigFloat(0.0)), 100000000))
print("Interval (-100000,0): ")
println(probability_satisfied(6, (BigFloat(-100000.0), BigFloat(0.0)), 100000000))
print("Interval (-1000000,0): ")
println(probability_satisfied(6, (BigFloat(-1000000.0), BigFloat(0.0)), 100000000))

Interval (-100,0): 8.0e-8
Interval (-1000,0): 1.6e-7
Interval (-10000,0): 1.3e-7
Interval (-100000,0): 1.6e-7
Interval (-1000000,0): 1.2e-7


In [13]:
print("Interval (-100,-1): ")
println(probability_satisfied(6, (BigFloat(-100.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000,-1): ")
println(probability_satisfied(6, (BigFloat(-1000.0), BigFloat(-1.0)), 100000000))
print("Interval (-10000,-1): ")
println(probability_satisfied(6, (BigFloat(-10000.0), BigFloat(-1.0)), 100000000))
print("Interval (-100000,-1): ")
println(probability_satisfied(6, (BigFloat(-100000.0), BigFloat(-1.0)), 100000000))
print("Interval (-1000000,-1): ")
println(probability_satisfied(6, (BigFloat(-1000000.0), BigFloat(-1.0)), 100000000))

Interval (-100,-1): 0.0
Interval (-1000,-1): 3.0e-8
Interval (-10000,-1): 9.0e-8
Interval (-100000,-1): 1.3e-7
Interval (-1000000,-1): 1.4e-7
